In [1]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='ctt2024',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = ''' CREATE TABLE IF NOT EXISTS Expresso_2024 (
                                    DATA_CRIACAO      text,
                                    CENTRO    text,
                                    Giro text,
                                    LOPTICA  text,
                                    JANELA_HORARIA text,
                                    NOME text,
                                    MORADA text,
                                    CP text,
                                    LOCALIDADE text,
                                    COD_T_EVEN text,
                                    DATA_EVENTO text,
                                    LATITUDE text,
                                    LONGITUDE text,
                                    NOME_REM text,
                                    COD_PAIS_ORIGEM text) '''
criar_db(sql)


In [2]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor()
cur.execute("SELECT DISTINCT CONCAT(SUBSTRING(DATA_CRIACAO, 7, 4),SUBSTRING(DATA_CRIACAO, 4, 2),SUBSTRING(DATA_CRIACAO, 1, 2)) AS ConvertedDate FROM Expresso_2024")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20240102' '20240103' '20240104' '20240105' '20240106' '20240107'
 '20240108' '20240109' '20240110' '20240111' '20240112' '20240113'
 '20240114' '20240115' '20240116' '20240117' '20240118' '20240119'
 '20240120' '20240121' '20240122' '20240123' '20240124' '20240125'
 '20240126' '20240127' '20240128' '20240129' '20240130' '20240131'
 '20240201' '20240202' '20240203' '20240204' '20240205' '20240206'
 '20240207' '20240208' '20240209' '20240210' '20240211' '20240212'
 '20240213' '20240214' '20240215' '20240216' '20240217' '20240218'
 '20240219' '20240220' '20240221' '20240222' '20240223' '20240224'
 '20240225' '20240226' '20240227' '20240228' '20240229' '20240301'
 '20240302' '20240303' '20240304' '20240305' '20240306' '20240307'
 '20240308' '20240309' '20240310' '20240311' '20240312' '20240313'
 '20240314' '20240315' '20240316' '20240317' '20240318' '20240319'
 '20240320' '20240321' '20240322' '20240323' '20240324' '20240325'
 '20240326' '20240327' '20240328' '20240329' '20240330' '20240

Expresso

In [3]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import time

current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/EventosPDA')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')
    return decoded_line.strip()



data_validation = []


cur = con.cursor()

cur.execute("SELECT distinct  SUBSTRING(DATA_CRIACAO, 7, 4) ||  SUBSTRING(DATA_CRIACAO, 4, 2) ||  SUBSTRING(DATA_CRIACAO, 1, 2) as dias FROM Expresso_2024 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

cur.close()

for file in files:
    start_time = time.time()
    if file.startswith("E"):
        date_str_s = file[14:22].replace("/", "")
        ano = file[14:18]
        date_str_s_array = np.array(date_str_s)
        
        if date_str_s not in data_validation and ano == "2024":
            
            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/EventosPDA')
            
            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            # Process the data line by line, excluding the last line
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line
            df = pd.read_csv(io.StringIO('\n'.join(data_rows)), sep=';', on_bad_lines='skip', encoding="Latin-1")
            ftp.quit()
            
            if len(data_rows)==1: 
                continue
            else:
                # Create a DataFrame from the cleaned data rows


                # Insert the data into the database
                for index, row in df.iterrows():
                    try:
                        cur = con.cursor()
                        # Replace "NaN" values with None (NULL) for database insertion
                        row = [None if pd.isna(value) else value for value in row]
                        placeholders = ', '.join(['%s'] * len(row))
                        insert_query = f"INSERT INTO Expresso_2024 ({', '.join(df.columns)}) VALUES ({placeholders})"
                        cur.execute(insert_query, tuple(row))
                    except (Exception, psycopg2.DatabaseError) as error:
                        print(f"Error inserting row {index+1}: {error}")
                        con.rollback()
                    else:
                        con.commit()
                        
                end_time = time.time()    
                print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")

con.close()


File 'EVPDA3466_ENV_20240622_20240624173621.CSV' inserted into the database: 2.136241912841797 seconds


BANCA 

In [4]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='ctt2024',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = ''' CREATE TABLE IF NOT EXISTS BANCA_2024 (
                                    COD_CLIENTE_BNC text,
                                    CODIGO_BALCAO_SIDIR text,
                                    CODIGO_BALCAO_AF text,
                                    AGENCIA text,
                                    CA text,
                                    CIRCUITO text,
                                    DATA text,
                                    CONTACTO text,
                                    FATURAR text,
                                    ENTREGUE text,
                                    TAREFA text,
                                    CP text,
                                    CENTRO_OPERACIONAL text,
                                    MORADA_TOQUE text,
                                    HORA_TOQUE_PREVISTA text,
                                    HORA_TOQUE_REAL text
                                    ) '''
criar_db(sql)





In [5]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute("SELECT distinct REPLACE(data, '.', '') dias FROM Banca_2024 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20240101' '20240102' '20240103' '20240104' '20240105' '20240108'
 '20240109' '20240110' '20240111' '20240112' '20240115' '20240116'
 '20240117' '20240118' '20240119' '20240122' '20240123' '20240124'
 '20240125' '20240126' '20240129' '20240130' '20240131' '20240201'
 '20240202' '20240205' '20240206' '20240207' '20240208' '20240209'
 '20240212' '20240213' '20240214' '20240215' '20240216' '20240219'
 '20240220' '20240221' '20240222' '20240223' '20240226' '20240227'
 '20240228' '20240229' '20240301' '20240304' '20240305' '20240306'
 '20240307' '20240308' '20240311' '20240312' '20240313' '20240314'
 '20240315' '20240318' '20240319' '20240320' '20240321' '20240322'
 '20240325' '20240326' '20240327' '20240328' '20240329' '20240401'
 '20240402' '20240403' '20240404' '20240405' '20240408' '20240409'
 '20240410' '20240411' '20240412' '20240415' '20240416' '20240417'
 '20240418' '20240419' '20240422' '20240423' '20240424' '20240425'
 '20240426' '20240429' '20240430' '20240501' '20240502' '20240

In [6]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import time

current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/Banca_Recolhas')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')
    return decoded_line.strip()

data_validation = []

con = psycopg2.connect(host='localhost', 
                    database='ctt2024',
                    user='postgres', 
                    password='postgres')

cur = con.cursor()

cur.execute("SELECT distinct REPLACE(data, '.', '') dias FROM Banca_2024 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

cur.close()

# Create a mapping between input column names and database column names
column_mapping = {
    'ï»¿COD_CLIENTE_BNC': 'COD_CLIENTE_BNC',
    'CODIGO_BALCAO_SIDIR': 'CODIGO_BALCAO_SIDIR',
    'CODIGO_BALCAO_AF': 'CODIGO_BALCAO_AF',
    'AGENCIA': 'AGENCIA',
    'CA': 'CA',
    'CIRCUITO': 'CIRCUITO',
    'DATA': 'DATA',
    'CONTACTO': 'CONTACTO',
    'FATURAR': 'FATURAR',
    'ENTREGUE': 'ENTREGUE',
    'TAREFA': 'TAREFA',
    'CP': 'CP',
    'CENTRO OPERACIONAL': 'CENTRO_OPERACIONAL',
    'MORADA TOQUE': 'MORADA_TOQUE',
    'HORA TOQUE PREVISTA': 'HORA_TOQUE_PREVISTA',
    'HORA TOQUE REAL': 'HORA_TOQUE_REAL'
}

for file in files:
    start_time = time.time()
    if file.startswith("C"):
        date_str_s = file[21:29].replace("/", "")
        date_str_s_array = np.array(date_str_s)
        ano = file[21:25]
        if date_str_s not in data_validation and ano =="2024":
            
            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/Banca_Recolhas')
            
            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            # Process the data line by line, excluding the last line
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line
            
            # Remove BOM if present
            if data_rows and data_rows[0].startswith('\ufeff'):
                data_rows[0] = data_rows[0][1:]
            
            # Join the data_rows without the last line (to exclude the empty line) and convert to bytes
            csv_data = '\n'.join(data_rows[:-1]).encode('utf-8')
            
            # Decode the bytes data using utf-8-sig to handle the BOM
            decoded_data = csv_data.decode('utf-8-sig')
            
            if decoded_data.strip():  # Check if the decoded data is not empty
                # Create a DataFrame from the cleaned data rows
                df = pd.read_csv(io.StringIO(decoded_data), sep=';', on_bad_lines='skip', encoding="Latin-1")

                # Rename the DataFrame columns based on the mapping
                df.rename(columns=column_mapping, inplace=True)

                # Insert the data into the database
                try:
                    cur = con.cursor()
                    # Replace "NaN" values with None (NULL) for database insertion
                    df = df.where(pd.notna(df), None)

                    # Generate placeholders for the query based on the number of columns
                    placeholders = ', '.join(['%s'] * len(df.columns))

                    # Construct the INSERT query with placeholders
                    insert_query = f"INSERT INTO Banca_2024 ({', '.join(df.columns)}) VALUES ({placeholders})"

                    # Execute the query with the DataFrame values as parameters
                    psycopg2.extras.execute_batch(cur, insert_query, df.values)
                    con.commit()

                    end_time = time.time()
                    print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")
                except (Exception, psycopg2.DatabaseError) as error:
                    print(f"Error inserting data from file '{file}': {error}")
                    con.rollback()

ftp.quit()
con.close()


File 'CAX019_TOQUE_BANCA_D_20240812_20240813000504.CSV' inserted into the database: 0.8345110416412354 seconds
File 'CAX019_TOQUE_BANCA_D_20240813_20240814000511.CSV' inserted into the database: 0.6749939918518066 seconds
File 'CAX019_TOQUE_BANCA_D_20240814_20240815000505.CSV' inserted into the database: 0.7196526527404785 seconds
File 'CAX019_TOQUE_BANCA_D_20240815_20240816000503.CSV' inserted into the database: 0.426929235458374 seconds
File 'CAX019_TOQUE_BANCA_D_20240816_20240817000507.CSV' inserted into the database: 0.7912302017211914 seconds
File 'CAX019_TOQUE_BANCA_D_20240819_20240820000506.CSV' inserted into the database: 0.805232048034668 seconds


Recolhas

In [7]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='ctt2024',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = '''CREATE TABLE IF NOT EXISTS Recolhas_2024 (
            "data_ficheiro" text,
            "Código Orgânico" text,
            "Descrição" text,
            "Dia" text,
            "Giro" text,
            "Tipo" text,
            "Ponto Recolha" text,
            "Nome Cliente" text,
            "Morada de recolha" text,
            "Código postal recolha" text,
            "Qntd de Objetos Recolhidos" text,
            "Estado" text,
            "Hora Início" text,
            "Hora Fim" text,
            "Hora Real" text
        )'''


criar_db(sql)



In [8]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)

cur.execute("SELECT DISTINCT  data_ficheiro as dias FROM Recolhas_2024  ORDER BY DIAS ASC")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20240101' '20240102' '20240103' '20240104' '20240105' '20240106'
 '20240107' '20240108' '20240109' '20240110' '20240111' '20240112'
 '20240113' '20240114' '20240115' '20240116' '20240117' '20240118'
 '20240119' '20240120' '20240121' '20240122' '20240123' '20240124'
 '20240125' '20240126' '20240127' '20240128' '20240129' '20240130'
 '20240131' '20240201' '20240202' '20240203' '20240204' '20240205'
 '20240206' '20240207' '20240208' '20240209' '20240210' '20240211'
 '20240212' '20240213' '20240214' '20240215' '20240216' '20240217'
 '20240218' '20240219' '20240220' '20240221' '20240222' '20240223'
 '20240224' '20240225' '20240226' '20240227' '20240228' '20240229'
 '20240301' '20240302' '20240303' '20240304' '20240305' '20240306'
 '20240307' '20240308' '20240309' '20240310' '20240311' '20240312'
 '20240313' '20240314' '20240315' '20240316' '20240317' '20240318'
 '20240319' '20240320' '20240321' '20240322' '20240323' '20240324'
 '20240325' '20240326' '20240327' '20240328' '20240329' '20240

In [9]:
import pandas as pd
import numpy as np
import io
import time
from ftplib import FTP
import psycopg2
import psycopg2.extras
import datetime


current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/Banca_Recolhas')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')
    return decoded_line.strip()

data_validation = []

con = psycopg2.connect(host='localhost', 
                    database='ctt2024',
                    user='postgres', 
                    password='postgres')

cur = con.cursor()

cur.execute("SELECT DISTINCT  data_ficheiro as dias FROM Recolhas_2024  ORDER BY DIAS ASC")

for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

# List of column names
columns = ['"Código Orgânico"', '"Descrição"', '"Dia"', '"Giro"', '"Tipo"', '"Ponto Recolha"', '"Nome Cliente"',
           '"Morada de recolha"', '"Código postal recolha"', '"Qntd de Objetos Recolhidos"', '"Estado"', '"Hora Início"',
           '"Hora Fim"', '"Hora Real"']

# Assuming you have a 'files' list defined earlier
for file in files:
    start_time = time.time()
    if file.startswith("R"):
        date_str_s = file[19:27].replace("/", "")
        date_str_s_array = np.array(date_str_s)
        ano = file[19:23]
        if date_str_s not in data_validation and ano == "2024":

            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/Banca_Recolhas')

            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            # Process the data line by line, excluding the last line
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line

            # Remove BOM if present
            if data_rows and data_rows[0].startswith('\ufeff'):
                data_rows[0] = data_rows[0][1:]

            # Join the data_rows without the last line (to exclude the empty line) and convert to bytes
            csv_data = '\n'.join(data_rows[:-1]).encode('utf-8')

            # Decode the bytes data using utf-8-sig to handle the BOM
            decoded_data = csv_data.decode('utf-8-sig')

            if decoded_data.strip():  # Check if the decoded data is not empty
                # Create a DataFrame from the cleaned data rows, specifying column names
                df = pd.read_csv(io.StringIO(decoded_data), sep=';', names=columns, on_bad_lines='skip', encoding="Latin-1")
                df = df.iloc[1:]

                # Add the 'data_ficheiro' column and fill it with date_str_s value
                df['data_ficheiro'] = date_str_s

                # Insert the data into the database
                try:
                    # Replace "NaN" values with None (NULL) for database insertion
                    df = df.where(pd.notna(df), None)

                    # Generate placeholders for the query based on the number of columns
                    placeholders = ', '.join(['%s'] * len(df.columns))

                    # Construct the INSERT query with placeholders
                    insert_query = f"INSERT INTO Recolhas_2024 ({', '.join(df.columns)}) VALUES ({placeholders})"

                    # Execute the query with the DataFrame values as parameters
                    psycopg2.extras.execute_batch(cur, insert_query, df.values)
                    con.commit()

                    end_time = time.time()
                    print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")
                except (Exception, psycopg2.DatabaseError) as error:
                    print(f"Error inserting data from file '{file}': {error}")
                    con.rollback()


ftp.quit()
con.close()


File 'RECSG4366_RECOLHAS_20240812_20240813010501.CSV' inserted into the database: 1.4212870597839355 seconds
File 'RECSG4366_RECOLHAS_20240813_20240814010500.CSV' inserted into the database: 2.059366226196289 seconds
File 'RECSG4366_RECOLHAS_20240814_20240815010500.CSV' inserted into the database: 1.3753368854522705 seconds
File 'RECSG4366_RECOLHAS_20240815_20240816010500.CSV' inserted into the database: 0.6690771579742432 seconds
File 'RECSG4366_RECOLHAS_20240816_20240818001006.CSV' inserted into the database: 0.710547924041748 seconds
File 'RECSG4366_RECOLHAS_20240817_20240818010500.CSV' inserted into the database: 0.6522409915924072 seconds
File 'RECSG4366_RECOLHAS_20240818_20240819010500.CSV' inserted into the database: 0.7545249462127686 seconds
File 'RECSG4366_RECOLHAS_20240819_20240820010500.CSV' inserted into the database: 1.433811902999878 seconds


REDE BASE


In [11]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='ctt2024',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = ''' CREATE TABLE IF NOT EXISTS REDE_BASE_2024 (
                                    DATA_CRIACAO      text,
                                    CENTRO    text,
                                    Giro text,
                                    LOPTICA  text,
                                    JANELA_HORARIA text,
                                    NOME text,
                                    MORADA text,
                                    CP text,
                                    LOCALIDADE text,
                                    COD_T_EVEN text,
                                    DATA_EVENTO text,
                                    LATITUDE text,
                                    LONGITUDE text,
                                    NOME_REM text,
                                    COD_PAIS_ORIGEM text) '''
criar_db(sql)


In [12]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute("SELECT distinct  SUBSTRING(DATA_CRIACAO, 7, 4) ||  SUBSTRING(DATA_CRIACAO, 4, 2) ||  SUBSTRING(DATA_CRIACAO, 1, 2) as dias FROM REDE_BASE_2024 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20240102' '20240103' '20240104' '20240105' '20240106' '20240108'
 '20240109' '20240110' '20240111' '20240112' '20240113' '20240114'
 '20240115' '20240116' '20240117' '20240118' '20240119' '20240120'
 '20240122' '20240123' '20240124' '20240125' '20240126' '20240127'
 '20240128' '20240129' '20240130' '20240131' '20240201' '20240202'
 '20240203' '20240204' '20240205' '20240206' '20240207' '20240208'
 '20240209' '20240210' '20240211' '20240212' '20240213' '20240214'
 '20240215' '20240216' '20240217' '20240219' '20240220' '20240221'
 '20240222' '20240223' '20240224' '20240225' '20240226' '20240227'
 '20240228' '20240229' '20240301' '20240302' '20240304' '20240305'
 '20240306' '20240307' '20240308' '20240309' '20240311' '20240312'
 '20240313' '20240314' '20240315' '20240316' '20240318' '20240319'
 '20240320' '20240321' '20240322' '20240323' '20240325' '20240326'
 '20240327' '20240328' '20240329' '20240330' '20240401' '20240402'
 '20240403' '20240404' '20240405' '20240406' '20240408' '20240

In [13]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import time

current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/EventosPDA_Rede_base')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')

    # Check if the number of columns is 13 (assuming semicolon as the delimiter)
    if len(decoded_line.split(';')) == 13:
        return decoded_line.strip()
    else:
        print(f"Ignoring row with incorrect number of columns: {decoded_line}")
        return None  # Skip the row if it has an incorrect number of columns

data_validation = []

cur = con.cursor()

cur.execute("SELECT distinct  SUBSTRING(DATA_CRIACAO, 7, 4) ||  SUBSTRING(DATA_CRIACAO, 4, 2) ||  SUBSTRING(DATA_CRIACAO, 1, 2) as dias FROM REDE_BASE_2024 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

cur.close()

for file in files:
    start_time = time.time()
    if file.startswith("E"):
        date_str_s = file[14:22].replace("/", "")
        month = file[18:20].replace("/", "")
        date_str_s = np.array(date_str_s)
        ano = file[14:18]
        if date_str_s not in data_validation and month >= "08" and ano == "2024":
            print(date_str_s)

            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/EventosPDA_Rede_base')

            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            try:
                df = pd.read_csv(file_data, sep=';', on_bad_lines='skip', encoding="Latin-1")
            except pd.errors.ParserError as pe:
                print(f"ParserError occurred while reading CSV in file '{file}':")
                print(pe)
                print(f"Skipping file '{file}'.")
                continue

            ftp.quit()

            if len(df) == 0:
                print(f"All rows in file '{file}' were problematic. Skipping insertion.")
                continue
            else:
                # Create a DataFrame from the cleaned data rows

                # Insert the data into the database
                for index, row in df.iterrows():
                    try:
                        cur = con.cursor()
                        # Replace "NaN" values with None (NULL) for database insertion
                        row = [None if pd.isna(value) else value for value in row]
                        placeholders = ', '.join(['%s'] * len(row))
                        insert_query = f"INSERT INTO REDE_BASE_2024 ({', '.join(df.columns)}) VALUES ({placeholders})"
                        cur.execute(insert_query, tuple(row))
                    except (Exception, psycopg2.DatabaseError) as error:
                        print(f"Error inserting row {index+1} from file '{file}': {error}")
                        con.rollback()
                    else:
                        con.commit()

                end_time = time.time()
                print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")

con.close()


20240804
File 'EVRBP4348_ENV_20240804_20240805010523.CSV' inserted into the database: 0.2836008071899414 seconds
20240811
File 'EVRBP4348_ENV_20240811_20240812010506.CSV' inserted into the database: 0.25863027572631836 seconds
20240812
File 'EVRBP4348_ENV_20240812_20240813010518.CSV' inserted into the database: 55.721471071243286 seconds
20240813
File 'EVRBP4348_ENV_20240813_20240814010501.CSV' inserted into the database: 55.51428985595703 seconds
20240814
File 'EVRBP4348_ENV_20240814_20240815010515.CSV' inserted into the database: 75.94307613372803 seconds
20240815
File 'EVRBP4348_ENV_20240815_20240816010524.CSV' inserted into the database: 1.0807209014892578 seconds
20240816
File 'EVRBP4348_ENV_20240816_20240817010513.CSV' inserted into the database: 62.69396996498108 seconds
20240817
File 'EVRBP4348_ENV_20240817_20240818010506.CSV' inserted into the database: 2.230713129043579 seconds
20240818
File 'EVRBP4348_ENV_20240818_20240819010501.CSV' inserted into the database: 0.27138495445

lista de tarefas

In [ ]:
# import psycopg2
# import psycopg2.extras
# import os



# def conecta_db():
#       con = psycopg2.connect(host='localhost', 
#                          database='ctt2024',
#                          user='postgres', 
#                          password='postgres')
#       return con


# def criar_db(sql):
#     con = conecta_db()
#     cur = con.cursor()
#     cur.execute(sql)
#     con.commit()
#     con.close()


# def inserir_db(sql):
#     con = conecta_db()
#     cur = con.cursor()
#     try:
#         cur.execute(sql)
#         con.commit()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print("Error: %s" % error)
#         con.rollback()
#         cur.close()
#         return 1
#     cur.close()

# sql = ''' CREATE TABLE IF NOT EXISTS Lista_tarefas_2023 (
#                                     LISTA_TAREFAS      text,
#                                     GIRO    text,
#                                     CENTRO text,
#                                     COLABORADOR  text,
#                                     CODIGO_SERVICO text,
#                                     DATA_HORA_ENTRADA_LT text,
#                                     TIPO_SERVICO text,
#                                     QUANTIDADE text,
#                                     NOME text,
#                                     ESTADO_SERVICO text,
#                                     DATA_SERVICO text,
#                                     DATA_FECHO_LT text,
#                                     EVENTO text,
#                                     RAZAO_SITUACAO text) '''
# criar_db(sql)


In [ ]:
# import numpy as np
# data_validation = []
# con = conecta_db()
# cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
# cur.execute("SELECT distinct  SUBSTRING(LISTA_TAREFAS, 1, 8) as dias FROM Lista_tarefas_2023 order by dias asc")
# for record in cur.fetchall():
#     if record[0] not in data_validation:
#         data_validation.append(record[0])
# cur.close()

# data_validation=np.array(data_validation)
# print(data_validation)

In [ ]:
# from ftplib import FTP
# import os
# import io
# import datetime
# import pandas as pd
# import numpy as np
# import psycopg2
# import psycopg2.extras
# import time

# current_year = datetime.datetime.now().year

# ftp = FTP()
# ftp.connect(host='10.0.25.193', port=2122)
# ftp.login(user='cax_sgc', passwd='gy768#lo')

# ftp.cwd('/OP/Rotas/Listas_Tarefas')

# files = ftp.nlst()


# def process_data(row):
#     try:
#         decoded_line = row.decode('Latin-1')
#     except UnicodeDecodeError as e:
#         print(f"UnicodeDecodeError in file '{file}', line: {row}")
#         decoded_line = row.decode('utf-8')
#     return decoded_line.strip()

# data_validation = []


# cur = con.cursor()

# cur.execute("SELECT distinct  SUBSTRING(LISTA_TAREFAS, 1, 8) as dias FROM Lista_tarefas_2023 order by dias asc")
# for record in cur.fetchall():
#     if record[0] not in data_validation:
#         data_validation.append(record[0])

# cur.close()

# for file in files:
    
#     start_time = time.time()
#     if file.startswith("L"):
#         Date_str_s = file[11:19].replace("/", "")
#         month= file[15:17].replace("/", "")
#         date_str_s= np.array(Date_str_s)
#         if date_str_s not in data_validation and month == "12":        
#             print(date_str_s)
#             ftp = FTP()
#             ftp.connect(host='10.0.25.193', port=2122)
#             ftp.login(user='cax_sgc', passwd='gy768#lo')
#             ftp.cwd('/OP/Rotas/Listas_Tarefas')
            
#             file_data = io.BytesIO()
#             ftp.retrbinary('RETR ' + file, file_data.write)
#             file_data.seek(0)  # Reset the stream position to the beginning

#             # Process the data line by line, excluding the last line
#             lines = file_data.readlines()
#             data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line
            
#             try:
#                 df = pd.read_csv(io.StringIO('\n'.join(data_rows)), sep=';')
#             except pd.errors.ParserError as pe:
#                 print(f"ParserError occurred while reading CSV in file '{file}':")
#                 print(pe)
#                 continue  
#             #df = pd.read_csv(io.StringIO('\n'.join(data_rows)), sep=';')
            
#             ftp.quit()
            
#             if len(data_rows)==1: 
#                 continue
#             else:
#                 # Create a DataFrame from the cleaned data rows


#                 # Insert the data into the database
#                 for index, row in df.iterrows():
#                     try:
#                         cur = con.cursor()
#                         # Replace "NaN" values with None (NULL) for database insertion
#                         row = [None if pd.isna(value) else value for value in row]
#                         placeholders = ', '.join(['%s'] * len(row))
#                         insert_query = f"INSERT INTO Lista_tarefas_2023 ({', '.join(df.columns)}) VALUES ({placeholders})"
#                         cur.execute(insert_query, tuple(row))
#                     except (Exception, psycopg2.DatabaseError) as error:
#                         print(f"Error inserting row {index+1}: {error}")
#                         con.rollback()
#                     else:
#                         con.commit()
                        
#                 end_time = time.time()    
#                 print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")


# con.close()
